## ОБЩИЙ ДЛЯ ВСЕХ РЕШЕНИЙ КОД
---

In [12]:
# НАСТРОЙКА JUPYTER LAB И ПОДКЛЮЧЕНИЕ БИБЛИОТЕК

# Импортировать библиотеки: numpy, pandas pandasql для работы с датасетами
import numpy as np 
import pandas as pd
import pandasql as ps

# Импортировать библиотеку datetime для работы с датой, временем
import datetime as dt

# Импортировать библиотеки pathlib и csv для работы с файлами и импорта csv
from pathlib import Path
import csv

# Импортировать библиотеки hashlib для работы с криптофункциями в том числе с 'md5'
import hashlib as hs

# Настройка среды Jupyter Lab
pd.options.display.max_rows = 400

# БЛОК ИМПОРТА ИСХОДНЫХ ВЕСОВЫХ ДАННЫХ

    # загружаем исходные данные из файла с записями с весовыми показателями и номерами вагонов;
    # чистим загруженные данные;
    # генерируем и добавляем производные данные о массе;
    # на выходе получаем таблицу 'scales'
# Подготовить путь к исходным CSV файлам
dir_path = Path.cwd()

# Загрузить данные из файла *scales*, файл содержит сырые данные по измеренным показателям массы
# номер состава, номер вагона, дата/время, и ряд друхих


# Подготовить переменную с именем импортируемого CSV файла с массами цистерн
file_name_scales = '01__snp-scales.csv'
path_file_scales = Path(dir_path,'res', 'data', file_name_scales)

# Загрузить таблицу с данными взвешивания
with open(path_file_scales, "r", encoding='utf-8') as csv_file_scales:
    scales = pd.read_csv(csv_file_scales,
                         delimiter=';',
                         header=0,
                         names=  ['trnum', 'num', 'bdatetime', 'invnum', 'tcalibr', 'tare', 'brutto', 'netto', 'velocity'] )

# Добавить новый индексный столбец 'id_scales'
scales['id_scales'] = np.arange(2, len(scales['invnum'])+2, 1)

# Зафиксировать количество записей полученных из CSV файла
len_scales = len(scales)

# Привести даты к типу datetime
scales['bdatetime'] = pd.to_datetime(scales['bdatetime'], dayfirst=True, format='%d.%m.%Y %H:%M:%S')

# ЧИСТИМ ДАННЫЕ
# Удалить не нужные столбцы: 'tcalibr', 'netto1', 'velocity'
scales.drop(['tcalibr', 'velocity'], axis=1, inplace=True)

# Удалить строки у которых значение столбца invnum = NaN
scales.dropna(subset=['invnum'], inplace=True)

# Заменить тип столбца 'invnum' на int64
scales['invnum'] = scales['invnum'].astype('int64')

# ГЕНЕРИРОВАТЬ И ДОБАВИТЬ ПРОИЗВОДНЫЕ ДАННЫЕ
# Добавить новый столбец 'deltaweight' содержащий разницу массы на въезде и на выезде
scales['deltaweight'] = scales['brutto'] - scales['tare']

# Задать новый порядок столбцов и проиндексировать
scales=scales.reindex(columns=['id_scales', 'trnum', 'invnum', 'num', 'bdatetime', 'tare', 'brutto', 'netto', 'deltaweight'])

# РАЗБИТЬ ПОЛУЧЕННЫЕ ДАННЫЕ ДВЕ ОСНОВНЫЕ РАБОЧИЕ ТАБЛИЦЫ

    # выделить набор scales_in - записи с составами ВЪЕХАВШИМИ на базу;
    # выделить набор scales_out - записи с составами ВЫЕХАВШИМИ с базы;
    # разделение провести по значению '0' в поле 'tare'
    # на выходе получаем таблицы 'scales_in' и 'scales_out'

# ВЫделить в отдельную таблицу 'scales_in' записи с транзакцией заехавших составов, по условию 'tare' = 0
scales_in = scales[ scales['tare'] == 0 ]
# ВЫделить в отдельную таблицу 'scales_out' записи с транзакцией выехавших составов, по условию 'tare' != 0
scales_out = scales[ scales['tare'] != 0 ]

# Задать индексацию по 'id_scales'
#scales_in = scales_in.set_index(keys = ['id_scales'], drop=False)
#scales_out = scales_out.set_index(keys = ['id_scales'], drop=False)

# БЛОК ИМПОРТА ИСХОДНЫХ ДАННЫХ ПО ОТБРАКОВКЕ

    # загружаем исходные данные из файла с записями о забракованных вагонов;
    # генерируем и добавляем поле 'id_marriag', которое позволит сопоставлять записи результатов работы программы с исходными 'грязными' данными до обработки
    # чистим загруженные данные;
    # на выходе получаем таблицу 'marriag'

# Загрузить данные из файла *marriag*, файл содержит сырые данные по отбракованным вагонам
# В итоге сосздается dataset 'marriag'

# Подготовить переменную с именем импортируемого CSV файла с дефектными цистернами
file_name_marriag = '02__snp-marriag.csv'
path_file_marriag = Path(dir_path,'res', 'data', file_name_marriag)

# Загрузить таблицу с данными отбраковки
with open(path_file_marriag, "r", encoding='utf-8') as csv_file_marriag:
    marriag = pd.read_csv(csv_file_marriag,
                          delimiter=';',
                          header=0,
                          names=  ['n', 'invnum', 'tcalibr', 'mass', 'nact', 'bdatetime', 'reason', 'contractor'])

# Добавить индексный столбец ID с индексом
marriag['id_marriag'] = np.arange(2, len(marriag['invnum'])+2, 1)

# Удалить не нужные столбцы
marriag.drop(['tcalibr', 'mass'], axis=1, inplace=True)

# Привести даты к типу datetime
marriag['bdatetime'] = pd.to_datetime(marriag['bdatetime'], dayfirst=True, format='%d.%m.%Y')

# Задать новый порядок столбцов и проиндексировать
marriag = marriag.reindex(columns=['invnum', 'id_marriag',  'bdatetime', 'n', 'nact', 'reason', 'contractor'])

# Задать индексацию по 'id_marriag'
marriag = marriag.set_index(keys = ['id_marriag'])

## ВЕТКА 1  
---
РЕШЕНИЕ ЗАДАЧИ ЧЕРЕЗ ЯВНО СУЩЕСТВУЮЩИЕ ПРИЗНАКИ. НЕ ВВОДИТСЯ И НЕ ИСПОЛЬЗУЕТСЯ ПОНЯТИЕ HASH ЗНАЧЕНИЯ

In [25]:
# БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_in' И 'scales_out'

# В данном блоке производится объединение записей таблиц 'scales_in' и 'scales_out' для получения 'правильных' записей о транзакции
# состава и всех входящих в него вагонов которые въехали на базу для налива и выехали после. Соединение производится по правилу LEFT JOIN
# с выполнением тройного условия. Так как 'pandas' не умеет выполнять оьбъединенеия по нечеткому соответствию (а именно таким получается
# третье условие), то для соединения была задействована дополнительная библиотека 'pandassql' которая работает поверх 'pandas' и перегоняет
# (динамичесуки) данные в SQL базу данных SQLite, и позволяет выполнить преобразование с помощью команд SQL, а после обратно сохранить в
# объекты 'pandas' - DataFrames.
# Условия соединенеия подразумевают совпадения следующих полей обеих DataFrames:

    # 'invnum';
    # 'num';
    # нечеткое совпадение временного поля 'bdatetime' (по принципу оконной функции);
    # на выходе получаем таблицу 'scales_join';

# Подготовить SQL запрос для левого соединения таблиц 'scales_in' и 'scales_out' с помощью функционала библиотеки 'pandasql'

join_query_scales = '''
    SELECT
        l.id_scales      as id_scales
        ,r.trnum         as trnum_in
        ,l.trnum         as trnum_out
        ,r.invnum        as invnum_in
        ,l.invnum        as invnum_out
        ,r.num           as num_in
        ,l.num           as num_out
        ,r.bdatetime     as date_in
        ,l.bdatetime     as date_out
        ,r.brutto        as tare_in
        ,l.tare          as tare_out
        ,l.brutto        as brutto
        ,l.netto         as netto
        ,l.deltaweight   as deltaweight
    FROM scales_in as r LEFT JOIN scales_out as l
         ON (r.invnum = l.invnum)
            AND (r.num = l.num)
            AND (cast(strftime('%s',r.bdatetime) as interger)
                BETWEEN cast(strftime('%s',l.bdatetime, '-37 hours') as interger) AND cast(strftime('%s',l.bdatetime) as interger) )
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
scales_join = ps.sqldf(join_query_scales, locals())

# Привести даты к типу datetime
scales_join['date_in'] = pd.to_datetime(scales_join['date_in'])
scales_join['date_out'] = pd.to_datetime(scales_join['date_out'])

# Вставить столбец 'deltatetime' с данными разницы времени (дени - часа - минуты - секунды) между событием 'въехал' и событием 'выехал'
scales_join.insert(9, 'deltatetime', scales_join['date_out']-scales_join['date_in'])

# Нормировать разницу к формату 00,00 (часы и доли часа)
scales_join['deltatetime'] = (scales_join['deltatetime']/pd.Timedelta('1 hour')).round(2)

# Преобразовать 'deltatetime' к string и заменить '.' на ',' т.к. импортирует с '.' асурд
scales_join['deltatetime'] = scales_join['deltatetime'].astype(str).str.replace('.', ',', regex=False)
scales_join['deltatetime'] = scales_join['deltatetime'].astype(str).str.replace('nan', '', regex=False)

# Заменить NaN в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на 0
scales_join[[ 'id_scales'
             ,'trnum_out'
             ,'invnum_out'
             ,'num_out'
             ,'deltatetime'
             ,'tare_out'
             ,'brutto'
             ,'netto'
             ,'deltaweight']] = scales_join[['id_scales', 'trnum_out', 'invnum_out', 'num_out' ,'deltatetime', 'tare_out', 'brutto', 'netto', 'deltaweight']].fillna(0)

# Заменить тип в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на int64
scales_join[[ 'id_scales'
             ,'trnum_out'
             ,'invnum_out'
             ,'num_out'
             ,'tare_out'
             ,'brutto'
             ,'netto'
             ,'deltaweight']] = scales_join[['id_scales', 'trnum_out', 'invnum_out', 'num_out', 'tare_out','brutto', 'netto', 'deltaweight']].astype('int64')

# БЛОК СОХРАНЕНИЯ ПРОМЕЖУТОЧНОГО ФАЙЛА CSV С ДАННЫМИ ТАБЛИЦЫ 'scales_join'
    # на выходе получаем CSV файл

# Подготовить переменную с именем экспортируемого CSV файла
file_name_scales_join_csv = '03__result-scales-correct-branch1.csv'
path_file_scales_join_csv = Path(dir_path,'res', 'data', file_name_scales_join_csv)

file_name_scales_join_excel = '03__result-scales-correct-branch1.xlsx'
path_file_scales_join_excel = Path(dir_path,'res', 'data', file_name_scales_join_excel)

# Записать данные в таблицу
scales_join.to_csv(str(path_file_scales_join_csv), index=False, sep=';', encoding='utf-8')
scales_join.to_excel(str(path_file_scales_join_excel), index=False)

# БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_join' И 'marriag' ДЛЯ ПОЛУЧЕНИЯ ТАБЛИЦЫ ЯВЛЯЮЩЕЙСЯ РЕШЕНИЕМ ЗАДАЧИ

# В данном блоке производится объединение записей таблиц scales_join и 'marriag' для получения итоговоЙ ТАБЛИЦЫ, которая является решением задачи.
# Позже на ее основе будет получен итоговый выходной файл правильных' записей о транзакции состава и всех входящих в него вагонов которые въехалина
# базу для налива и выехали после. Соединение производится по правилу LEFT JOIN с выполнением двойного условия. Так как 'pandas' не умеет выполнять
# объединенеия по нечеткому соответствию (а именно таким получается второе условие), то для соединения была задействована дополнительная библиотека
# 'pandassql' которая работает поверх 'pandas' и перегоняет (динамически) данные в SQL базу данных SQLite, далее позволяет выполнить преобразование
# с помощью команд SQL, а после обратно сохранить в объекты 'pandas' - DataFrames. Условия соединенеия подразумевают совпадения следующих полей обеих таблиц:

    # 'invnum_out' и 'invnum';
    # нечеткое совпадение временного поля 'bdatetime' (по принципу оконной функции);
    # на выходе получаем таблицу 'join_hash_table';


# Подготовить SQL запрос для левого соединения таблиц 'scales_join' и 'marriag'

join_query_result_table = '''
    SELECT
        ROW_NUMBER() OVER(ORDER BY l.id_scales) + 1 as id_result
        ,l.id_scales   as id_scales
        ,r.id_marriag  as id_marriag
        ,l.trnum_in    as trnum_in
        ,l.trnum_out   as trnum_out
        ,l.invnum_in   as invnum_in
        ,l.invnum_out  as invnum_out
        ,l.num_in      as num_in
        ,l.num_out     as num_out
        ,l.date_in     as date_in
        ,l.date_out    as date_out
        ,r.bdatetime   as date_marriag
        ,l.deltatetime as deltatetime
        ,l.tare_in     as tare_in
        ,l.tare_out    as tare_out
        ,l.brutto      as brutto
        ,l.netto       as netto
        ,l.deltaweight as deltaweight
        ,r.nact        as nact
        ,r.reason      as reason
    FROM scales_join as l LEFT JOIN marriag as r
         ON (l.invnum_out = r.invnum)
            AND (cast(strftime('%s',r.bdatetime) as interger)
                BETWEEN cast(strftime('%s',l.date_out, '-30 hours') as interger) AND cast(strftime('%s',l.date_out) as interger) )        
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
join_table = ps.sqldf(join_query_result_table, locals())

# Привести даты к типу datetime
join_table['date_in'] = pd.to_datetime(join_table['date_in'])
join_table['date_out'] = pd.to_datetime(join_table['date_out'])
join_table['date_marriag'] = pd.to_datetime(join_table['date_marriag'])

# Заменить NaN в столбце 'id_marriag' на 0
join_table['id_marriag'] = join_table['id_marriag'].fillna(0)

# Заменить тип столбца 'id_marriag' на int64
join_table['id_marriag'] = join_table['id_marriag'].astype('int64')

# Заменить None в столбце 'nact' на 0
join_table['nact'] = join_table['nact'].fillna(0)

# Заменить тип столбца 'nact' на int64
join_table['nact'] = join_table['nact'].astype('int64')


# БЛОК СОХРАНЕНИЯ РЕШЕНИЯ ЗАДАЧИ ПО МЕТОДУ 'ВЕТКА 1' В CSV ФАЙЛ
    # на выходе получаем CSV файл сгенерированный в соответствии с логикой 'ВЕТКА 1'

# Подготовить переменную с именем результирующего CSV файла
file_name_join = '05__result-table-branch1.csv'
path_file_join = Path(dir_path,'res', 'data', file_name_join)

file_name_join_excel = '05__result-table-branch1.xlsx'
path_file_join_excel = Path(dir_path,'res', 'data', file_name_join_excel)

# Записать в данные в файл
join_table.to_csv(str(path_file_join), index=False, sep=';', encoding='utf-8')
join_table.to_excel(str(path_file_join_excel), index=False)

## ВЕТКА 2  
---
РЕШЕНИЕ ЗАДАЧИ ЧЕРЕЗ НЕ ЯВНО СУЩЕСТВУЮЩИЕ ПРИЗНАКИ. ДЛЯ РЕШЕНИЯ ВВОДИТСЯ И ИСПОЛЬЗУЕТСЯ ПОНЯТИЕ HASH ЗНАЧЕНИЯ

In [24]:
# ГРУППИРОВКА ДАННЫХ В DATAFRAMES scales_in И scales_out

# В данном блоке производится группировка записей таблиц 'scales_in' и 'scales_out' для получения сгруппированных по значению номера состава
# 'trnum' записей. Это нужно для достижения нескольких целей:

    # определения количества составов отдельно в таблицах въехавших и выехавших составов;
    # подготовка данных в обеих таблицах к такой форме хранения которая позволит эффективно вычислить hash значения;
    # вычисления комплексных характеристик каждого состава в обеих таблицах - hash значения;

# Выработка требований к hash значению - нам потребуется определить такую группу характеристик которая атомарно представляет уникальную
# характеристику состава, причем должна имется 100% возможность получить такую группу как в таблице въехавших составах так и в таблице
# выехавших составов. Это требование должно не укоснительно выполняться, так как в дальнейшем полученный слепок состава - hash значение
# должно стать идентификатором по которому будет искаться совпадение и далее проводиться объединение данных по идентичным составам из обеих таблиц.
# В качестве такого группового признака который имперически определен как абсолютно не повторяемый ни при каких условиях - последовательность
# значений номеров состава а также (одновременно) позиционный номер вагона в составе, т.е. математическая последовательность значений номеров вагонов.
# Для упрощения кода, фактор порядкового номера был заменен на не явный, а именно перед вычислением hash значения номера вагонов превращены в текстовые
# значения и произведена конкатенация строго в той последовательности в которой вагоны шли в составе. В итоге входным параметром функции-генератора
# hash значения стала тестовая строка состаящая из непрерывно сцепленных значений текстовых эквивалентов номеров вагонов.
# В качестве функции-генератора hash значения применена - MD5


# Вывести первые 3 записи (до группировки) комплементарных групп которе далее должны сгруппироваться а далее объединиться
# это требуется для визуального контроля
scales_in[ scales_in['trnum'] == 683482 ].sort_values(by='num').head(3)

# Произвести группировку по номеру состава ('trnum')
gr_scales_in = scales_in.groupby('trnum')
gr_scales_out = scales_out.groupby('trnum')

# БЛОК ГЕНЕРАЦИИ HASH ЗНАЧЕНИЙ

# Функция вычисляет hash значение группы
def HashRow(_df):
    
    hash_total_str = ''
    
    df = _df.sort_values(by='num', ascending=True)
    # Перебрать все строки текущей группы, получить текстовый объект hash всех номеров вагонов группы
    for data in df.itertuples():
        hash_total_str += str(data[3])
    
    hash_object = hs.md5(hash_total_str.encode('utf-8'))
    #return int(hash_object.hexdigest(), 16)
    return hash_object.hexdigest()

# Функция выпоняет перебор всех групп и вызывает функцию вычисления hash значения группы
def HashGroup(_df):
    
    hash_dict = {} # Словарь хранящий пару 'Номер состава: hash состава'
    cycle = 0
    
    # Перебраить все группы сгруппированого DataFrame
    for name, group in _df:
        if cycle>0:
            break
        hash_dict[name] = HashRow(group) # Для каждой группы вызвать hash функцию
    return hash_dict

# Сгенерировать словари с hash значениями групп составовs
hash_dict_in  = HashGroup(gr_scales_in)
hash_dict_out = HashGroup(gr_scales_out)

# Преобразовать dictonari в DataFrame
hash_df_in = pd.DataFrame(hash_dict_in.items(), columns=['trnum', 'hash_group'])
hash_df_out = pd.DataFrame(hash_dict_out.items(), columns=['trnum', 'hash_group'])

# БЛОК СОЕДИНЕНИЯ ТАБЛИЦ 'scales_in','scales_out' С СООТВЕТСТВУЮЩИМИ ТАБЛИЦАМИ HASH ЗНАЧЕНИЙ

# Выполнить левое соединение таблиц 'scales_hash_in', 'scales_hash_out' с соответствующими таблицами hash значений
scales_hash_in = scales_in.merge(hash_df_in, on='trnum', how='left')
scales_hash_out = scales_out.merge(hash_df_out, on='trnum', how='left')

# БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_hash_in' И 'scales_hash_out' ДЛЯ ПОЛУЧЕНИЯ ТАБЛИЦЫ 'scales_join_hash'
# КОТОРАЯ ХРАНИТ ОБЪЕДИНЕНЫЕ ЗАПИСИ ВЪЕХАВШИХ И ВЫЕХАВШИХ СОСТАВОВ.

# АНАЛОГИЧНАЯ ТАБЛИЦА ПОЛУЧЕНА РАНЕЕ В ВЕТКЕ 1 ПО ОТЛИЧНОМУ ОТ ВЕТКЕ 2 АЛГОРИТМУ, А ИМЕННО БЕЗ ИСПОЛЬЗОВАНИЯ HASH ЗНАЧЕНИЙ

# В данном блоке производится соединение записей таблиц 'scales_hash_in' и 'scales_hash_out'. Позже на основе результирующей
# таблицы 'scales_join_hash' будет записан результирующий выходной файл CSV файл, являющийся решением задачи.
# Соединение производится по правилу LEFT JOIN с выполнением тройного условия. В данном случае у нас все три условия четкие,
# поэтому можно произвести объединение средствами библиотеки 'pandas', но из за экономии времени, единообразности технических
# решений ранее уже примененных в данной программе, а также гибкости компоновки сстолбцов итоговой таблицы, соединение будет
# производится средствами библиотеки 'pandassql', которая работает поверх 'pandas' и перегоняет (динамически) данные в SQL базу
# данных SQLite, далее позволяет выполнить преобразование с помощью команд SQL, а после обратно сохранить в объекты 'pandas' - DataFrames.
# Условия соединенеия подразумевают совпадения следующих полей обеих таблиц:

    # 'invnum_out' и 'invnum';
    # 'num' обеих таблиц;
    # 'hash_group' обеих таблиц;
    # на выходе получаем таблицу 'scales_join_hash';

    # Преобразуем тип 'hash_group' в string, потому что pandassql не умеет конвертировать int large в целочисленные типы SQLite

# Данное действие требуется только для варианта программы где 'hash' значение конвертировалось в int larg, если работа ведется 
# с текстовыми 'hash' значениями то оно не нужно, поэтому эти строки кода могут быть закоментированы

#scales_hash_in['hash_group'] = scales_hash_in['hash_group'].astype(str)
#scales_hash_out['hash_group'] = scales_hash_out['hash_group'].astype(str)

# Подготовить SQL запрос для левого соединения таблиц 'scales_hash_in' и 'scales_hash_out'
join_query_scales_hash = '''
    SELECT
        l.id_scales     as id_scales
        ,l.trnum         as trnum_in
        ,r.trnum         as trnum_out
        ,l.invnum        as invnum_in
        ,r.invnum        as invnum_out
        ,l.num           as num_in
        ,l.num           as num_out
        ,l.bdatetime     as date_in
        ,r.bdatetime     as date_out
        ,l.brutto        as tare_in
        ,r.tare          as tare_out
        ,r.brutto        as brutto
        ,r.netto         as netto
        ,r.deltaweight   as deltaweight
        ,l.hash_group    as hash_group
    FROM scales_hash_in as l LEFT JOIN scales_hash_out as r
         ON     (l.invnum     = r.invnum)
            AND (l.num        = r.num)
            AND (l.hash_group = r.hash_group)
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
scales_join_hash = ps.sqldf(join_query_scales_hash, locals())

# Привести даты к типу datetime
scales_join_hash['date_in']  = pd.to_datetime(scales_join_hash['date_in'])
scales_join_hash['date_out'] = pd.to_datetime(scales_join_hash['date_out'])

# Вставить столбец 'deltatetime' с данными разницы времени (дени - часа - минуты - секунды) между событием 'въехал' и событием 'выехал'
scales_join_hash.insert(9, 'deltatetime', scales_join_hash['date_out']-scales_join_hash['date_in'])

# Нормировать разницу к формату 00,00 (часы и доли часа)
scales_join_hash['deltatetime'] = (scales_join_hash['deltatetime']/pd.Timedelta('1 hour')).round(2)

# Преобразовать 'deltatetime' к string и заменить '.' на ',' т.к. импортирует с '.' для дальнейшего использования совместно с Excel
scales_join_hash['deltatetime'] = scales_join_hash['deltatetime'].astype(str).str.replace('.', ',', regex=False)

# Заменить NaN в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на 0
scales_join_hash[['trnum_out'
                 ,'invnum_out'
                 ,'num_out'
                 ,'deltatetime'
                 ,'tare_out'
                 ,'brutto'
                 ,'netto'
                 ,'deltaweight']] = scales_join_hash[['trnum_out', 'invnum_out', 'num_out' ,'deltatetime', 'tare_out', 'brutto', 'netto', 'deltaweight']].fillna(0)

# Заменить тип в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на int64
scales_join_hash[['trnum_out'
                 ,'invnum_out'
                 ,'num_out'
                 ,'tare_out'
                 ,'brutto'
                 ,'netto'
                 ,'deltaweight']] = scales_join_hash[['trnum_out', 'invnum_out', 'num_out', 'tare_out','brutto', 'netto', 'deltaweight']].astype('int64')

# БЛОК СОХРАНЕНИЯ ПРОМЕЖУТОЧНОГО ФАЙЛА CSV С ДАННЫМИ ТАБЛИЦЫ 'scales_join'
    # на выходе получаем CSV файл сгенерированный в соответствии с логикой 'ВЕТКА 2'

# Подготовить переменную с именем экспортируемого CSV файла 
file_name_scales_join_hash_csv = '04__result-scales-correct-branch2.csv'
path_file_scales_join_hash_csv = Path(dir_path,'res', 'data', file_name_scales_join_hash_csv)

file_name_scales_join_hash_excel = '04__result-scales-correct-branch2.xlsx'
path_file_scales_join_hash_excel = Path(dir_path,'res', 'data', file_name_scales_join_hash_excel)

# Записать в данные в таблицу
scales_join_hash.to_csv(str(path_file_scales_join_hash_csv), index=False, sep=';', encoding='utf-8')
scales_join_hash.to_excel(str(path_file_scales_join_hash_excel), index=False)

# БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_join_hash' И 'marriag' ДЛЯ ПОЛУЧЕНИЯ ТАБЛИЦЫ ЯВЛЯЮЩЕЙСЯ РЕШЕНИЕМ ЗАДАЧИ
# В данном блоке производится объединение записей таблиц scales_join_hash и 'marriag' для получения итоговоЙ ТАБЛИЦЫ,
# которая является решением задачи. Позже на ее основе будет получен итоговый выходной файл правильных' записей о транзакции
# состава и всех входящих в него вагонов которые въехали на базу для налива и выехали после. Соединение производится по правилу
# LEFT JOIN с выполнением двойного условия. Так как 'pandas' не умеет выполнять оьбъединенеия по нечеткому соответствию (а именно
# таким получается второе условие), то для соединения была задействована дополнительная библиотека 'pandassql' которая работает
# поверх 'pandas' и перегоняет (динамически) данные в SQL базу данных SQLite, далее позволяет выполнить преобразование с помощью
# команд SQL, а после обратно сохранить в объекты 'pandas' - DataFrames. Условия соединенеия подразумевают совпадения следующих
# полей обеих таблиц:

    # 'invnum_out' и 'invnum';
    # нечеткое совпадение временного поля 'bdatetime' (по принципу оконной функции);
    # на выходе получаем таблицу 'join_table';

# Подготовить SQL запрос для левого соединения таблиц 'scales_join_hash' и 'marriag'

join_hash_query_result_table = '''
    SELECT
        ROW_NUMBER() OVER(ORDER BY l.id_scales) + 1 as id_result
        ,l.id_scales   as id_scales
        ,r.id_marriag  as id_marriag
        ,l.trnum_in    as trnum_in
        ,l.trnum_out   as trnum_out
        ,l.invnum_in   as invnum_in
        ,l.invnum_out  as invnum_out
        ,l.num_in      as num_in
        ,l.num_out     as num_out
        ,l.date_in     as date_in
        ,l.date_out    as date_out
        ,r.bdatetime   as date_marriag
        ,l.deltatetime as deltatetime
        ,l.tare_in     as tare_in
        ,l.tare_out    as tare_out
        ,l.brutto      as brutto
        ,l.netto       as netto
        ,l.deltaweight as deltaweight
        ,r.nact        as nact
        ,r.reason      as reason
        ,l.hash_group  as hash_group
    FROM scales_join_hash as l LEFT JOIN marriag as r
         ON (l.invnum_out = r.invnum)
            AND (cast(strftime('%s',r.bdatetime) as interger)
                BETWEEN cast(strftime('%s',l.date_out, '-30 hours') as interger) AND cast(strftime('%s',l.date_out) as interger) )        
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
join_hash_table = ps.sqldf(join_hash_query_result_table, locals())

# Привести даты к типу datetime
join_hash_table['date_in'] = pd.to_datetime(join_hash_table['date_in'])
join_hash_table['date_out'] = pd.to_datetime(join_hash_table['date_out'])
join_hash_table['date_marriag'] = pd.to_datetime(join_hash_table['date_marriag'])

# Заменить NaN в столбце 'id_marriag' на 0
join_hash_table['id_marriag'] = join_hash_table['id_marriag'].fillna(0)

# Заменить тип столбца 'id_marriag' на int64
join_hash_table['id_marriag'] = join_hash_table['id_marriag'].astype('int64')

# Заменить None в столбце 'nact' на 0
join_hash_table['nact'] = join_hash_table['nact'].fillna(0)

# Заменить тип столбца 'nact' на int64
join_hash_table['nact'] = join_hash_table['nact'].astype('int64')


# БЛОК СОХРАНЕНИЯ РЕШЕНИЯ ЗАДАЧИ ПО МЕТОДУ 'ВЕТКА 2' В CSV ФАЙЛ

# Подготовить переменную с именем результирующего CSV файла
file_name_join_hash_table_csv = '06__result-table-branch2.csv'
path_file_join_hash_table_csv = Path(dir_path,'res', 'data', file_name_join_hash_table_csv)

file_name_join_hash_table_excel = '06__result-table-branch2.xlsx'
path_file_join_hash_table_excel = Path(dir_path,'res', 'data', file_name_join_hash_table_excel)

# Записать в данные в файл
join_hash_table.to_csv(str(path_file_join_hash_table_csv), index=False, sep=';', encoding='utf-8')
join_hash_table.to_excel(str(path_file_join_hash_table_excel), index=False)